In [4]:
import sys
import talib as talib
import pandas as pd
import numpy as np
from pyecharts import options as opts
sys.path.append("..") 
from imp import reload
from src.ts_stock import TsStock
from src.chart import Chart
import tushare as ts
tsStock = TsStock();
chart = Chart();
token='36d50a0386e9066b5c79c2de58ea2d3a9b980020041adc910f76bfc7'
ts.set_token(token)
pro=ts.pro_api()


In [7]:
stock = '000001'
start_date = '20080101'
end_date=''

In [8]:
stock_info = tsStock.get_stock_info(stock)
df_income = pro.income(ts_code=stock_info['code'], start_date=start_date, end_date=end_date)
#df_income.to_excel("income.xlsx",sheet_name='Sheet_name_1')
df_income.index = pd.to_datetime( df_income.ann_date )
df_income = df_income.sort_index(ascending=True)

In [9]:
c = chart.get_line_income(df_income,stock_info)
c.render_notebook()